'''<br>
@Author: Rahul <br>
@Date: 2024-10-24<br>
@Last Modified by: Rahul <br>
@Last Modified time: 2024-10-24<br>
@Title: Python program demonstrate Import and Export DynamoDB<br>
'''

In [11]:
import boto3
import json
import csv
from decimal import Decimal

Import from DynamoDB as CSV file

In [9]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('Movies')

response = table.scan()
data = response['Items']

In [5]:
csv_file_name = 'data.csv'

if data:
    headers = data[0].keys()
else:
    headers = []

with open(csv_file_name, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=headers)
    writer.writeheader()  
    for item in data:
        writer.writerow(item)  

print(f"Data exported to {csv_file_name} successfully.")


Data exported to data.csv successfully.


Import from DynamoDB as JSON file

In [12]:
import json
from decimal import Decimal

def decimal_default(obj):
    if isinstance(obj, Decimal):
        return float(obj)  
    raise TypeError

json_file_name = 'data.json'  

# Write data to a JSON file
with open(json_file_name, 'w') as json_file:
    json.dump(data, json_file, default=decimal_default, indent=4)

print(f"Data exported to {json_file_name} successfully.")


Data exported to data.json successfully.


Export to DynamoDB

In [8]:

# Read data from the CSV file
with open('data.csv', mode='r') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        # Convert the year to an integer
        if 'year' in row:
            row['year'] = int(row['year'])  # Convert to integer
        
        # Assuming the CSV has columns that match the DynamoDB table's attributes
        table.put_item(Item=row)
        print(f"Inserted: {row}")  # Optional: print the inserted item


Inserted: {'year': 2019, 'plot': 'Action', 'rating': '8.5', 'title': 'Vikarm'}
Inserted: {'year': 2019, 'plot': 'Sci-Fi', 'rating': '9.5', 'title': 'Instellar'}
Inserted: {'year': 2019, 'plot': 'Comedy', 'rating': '7.5', 'title': 'Avesham'}
Inserted: {'year': 2019, 'plot': 'Rom-Com', 'rating': '8.5', 'title': 'Premalu'}


In [13]:

# Initialize the DynamoDB and S3 resources
dynamodb = boto3.resource('dynamodb')
s3 = boto3.client('s3')

# Specify the table name and the S3 bucket and file details
table_name = 'Movies'
bucket_name = 'rahulbllamdabucket'
s3_json_file = 'exported_data.json'  # S3 JSON file name
s3_csv_file = 'exported_data.csv'  # S3 CSV file name

# Function to handle Decimal types during JSON serialization
def decimal_default(obj):
    if isinstance(obj, Decimal):
        return float(obj)
    raise TypeError

# Export DynamoDB data to a JSON file and upload to S3
def export_to_s3_json():
    table = dynamodb.Table(table_name)
    
    # Scan DynamoDB table to get all data
    response = table.scan()
    data = response['Items']
    
    # Convert the data to a JSON format
    json_data = json.dumps(data, default=decimal_default, indent=4)
    
    # Upload JSON data to S3
    s3.put_object(Bucket=bucket_name, Key=s3_json_file, Body=json_data)
    print(f"Data exported to S3: {bucket_name}/{s3_json_file}")

# Export DynamoDB data to a CSV file and upload to S3
def export_to_s3_csv():
    table = dynamodb.Table(table_name)
    
    # Scan DynamoDB table to get all data
    response = table.scan()
    data = response['Items']
    
    # Define the CSV headers based on the keys in your data
    headers = data[0].keys() if data else []
    
    # Write data to CSV
    csv_file_name = 'local_temp_file.csv'
    with open(csv_file_name, 'w', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=headers)
        writer.writeheader()
        for item in data:
            writer.writerow({k: str(v) if isinstance(v, Decimal) else v for k, v in item.items()})
    
    # Upload CSV file to S3
    with open(csv_file_name, 'rb') as file_data:
        s3.upload_fileobj(file_data, bucket_name, s3_csv_file)
    print(f"Data exported to S3: {bucket_name}/{s3_csv_file}")

if __name__ == "__main__":
    # Choose either JSON or CSV export
    export_to_s3_json()  # Exports as JSON
    export_to_s3_csv()   # Exports as CSV


Data exported to S3: rahulbllamdabucket/exported_data.json
Data exported to S3: rahulbllamdabucket/exported_data.csv
